In [138]:
# Train a freakish classifier should not be that hard .....(Face Palm)

# Build Embedding Model to classify OOC behaviors, using provided dataset 
from database.pro.clean import clean_response
import glob 
import datasets 
from datasets import Dataset
from sklearn.model_selection import train_test_split


txt_files = glob.glob("database/pro/transcript/transcript_*.txt")
label_map = {0: "Agent", 1: "Customer"}




### 1. Naive Bayes gives 93%+ Accuracy

In [139]:
train_dataset, test_dataset = prepare_ooc_dataset(txt_files, cases)
vectorizer, nb_classifier = train_nb(train_dataset, test_dataset)


def check_performance(vectorizer, nb_classifier, cases, fn):
    error_count = 0
    for (text, label) in cases:
        (pred, confidence) = fn(text, vectorizer, nb_classifier)
        gt = label_map[label]  # Assuming all samples are "Agent" class
        error = pred != gt
        if error:
            error_count += 1
            print("Text: ", text, " | Prediction:", pred," | Confidence: ", confidence, "| Ground Truth:", gt)


check_performance(vectorizer, nb_classifier, cases, fn=pred_nb)

Accuracy: 0.91

Classification Report:
              precision    recall  f1-score   support

       Agent       0.93      0.87      0.90       456
    Customer       0.90      0.95      0.92       567

    accuracy                           0.91      1023
   macro avg       0.92      0.91      0.91      1023
weighted avg       0.91      0.91      0.91      1023

Text:  Have you heard about FWD before?  | Prediction: Customer  | Confidence:  0.5984111122132806 | Ground Truth: Agent
Text:  I work at FWD insurance  | Prediction: Customer  | Confidence:  0.8566138590075342 | Ground Truth: Agent


### 2. Classifier with Embedding models

In [143]:
tokenizer, model = train_em(train_dataset, test_dataset)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/2: 100%|██████████| 288/288 [02:59<00:00,  1.60it/s]


Epoch 1/2, Average Loss: 0.1367


Evaluating: 100%|██████████| 32/32 [00:05<00:00,  5.84it/s]


Accuracy: 0.9668

Classification Report:
              precision    recall  f1-score   support

       Agent       0.97      0.96      0.96       456
    Customer       0.97      0.97      0.97       567

    accuracy                           0.97      1023
   macro avg       0.97      0.97      0.97      1023
weighted avg       0.97      0.97      0.97      1023



Epoch 2/2: 100%|██████████| 288/288 [03:00<00:00,  1.60it/s]


Epoch 2/2, Average Loss: 0.0463


Evaluating: 100%|██████████| 32/32 [00:05<00:00,  5.61it/s]

Accuracy: 0.9677

Classification Report:
              precision    recall  f1-score   support

       Agent       0.94      0.99      0.96       456
    Customer       0.99      0.95      0.97       567

    accuracy                           0.97      1023
   macro avg       0.97      0.97      0.97      1023
weighted avg       0.97      0.97      0.97      1023



In [151]:
# The push_to_hub function cannot be imported from huggingface_hub
# If you need to upload the model to Hugging Face Hub, consider using
# the `transformers` library's built-in methods or the `huggingface_hub`
# library's alternative functions.

# For example, you might use:
from transformers import AutoModel, AutoTokenizer
model.push_to_hub("Ksgk-fy/ooc_patch_v1")
tokenizer.push_to_hub("Ksgk-fy/ooc_patch_v1")

# Or, if you prefer to use huggingface_hub directly:
# from huggingface_hub import HfApi
# api = HfApi()
# api.upload_file(
#     path_or_fileobj="./patch_model/pytorch_model.bin",
#     path_in_repo="pytorch_model.bin",
#     repo_id="Ksgk-fy/ooc_patch_v1",
# )
# Save the model and tokenizer locally
# model.save_pretrained("./patch_model")
# tokenizer.save_pretrained("./patch_model")

# # Upload to Hugging Face Hub
# push_to_hub(repo_id="Ksgk-fy/ooc_patch_v1", local_dir="./patch_model", use_auth_token=True)

# print("Model and tokenizer uploaded to Hugging Face Hub successfully!")


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Ksgk-fy/ooc_patch_v1/commit/1d0926245491b21f2e2aeaff2c1e53ec43e0da88', commit_message='Upload tokenizer', commit_description='', oid='1d0926245491b21f2e2aeaff2c1e53ec43e0da88', pr_url=None, pr_revision=None, pr_num=None)

In [1]:


# Move the model to the appropriate device (CPU or GPU)



/opt/homebrew/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

/opt/homebrew/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

NameError: name 'torch' is not defined

In [10]:
import torch 

repo_id = "Ksgk-fy/ooc_patch_v1"
# Load the model and tokenizer from the Hugging Face Hub
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(repo_id)

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(repo_id)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

label_map = {0: "Agent", 1: "Customer"}


def predict_em(sample_text, model, tokenizer, threshold_agent=0, threshold_customer=0):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    # Tokenize the sample text
    encoded = tokenizer(sample_text, padding=True, truncation=True, return_tensors='pt')
    input_ids = encoded['input_ids'].to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()
        predicted_confidence = torch.nn.functional.softmax(logits, dim=1)[0][predicted_class].item()

    # Map the predicted class to the corresponding label
    label_map = {0: "Agent", 1: "Customer"}
    thres_map = {0: threshold_agent, 1: threshold_customer}
    if predicted_confidence < thres_map[predicted_class]:
        return "Not Sure", 0
    else:
        predicted_label = label_map[predicted_class]
        return predicted_label, predicted_confidence


In [11]:
# Set the model to evaluation mode
model.eval()

cases = [    
    ("Have you heard of FWD before?", 0),
    ("I am not interested in your insurance product", 1),
    ("So what is the different between FWD insurance and others?", 1),
    ("What is FWD again?", 1),
    ("I do not trust FWD.", 1),
    ("Hello Alex!", 1),
    ("Can you explain the benefits of FWD insurance?", 1),
    ("I'm looking for the best insurance coverage.", 1),
    ("FWD seems like a reliable company.", 1),
    ("What are the premiums for FWD insurance?", 1),
    ("I've had bad experiences with insurance companies before.", 1),
    ("Is FWD insurance available in my country?", 1),
    ("How does FWD compare to other major insurers?", 1),
    ("I'm satisfied with my current insurance provider.", 1),
    ("Tell me more about FWD's customer service.", 1)
]



def check_performance(model, tokenizer, cases, fn=predict_em):
    error_count = 0
    for (text, label) in cases:
        (pred, confidence) = fn(text, model, tokenizer, 0.8, 0.)
        gt = label_map[label]  # Assuming all samples are "Agent" class
        error = pred != gt
        if error:
            error_count += 1
            print("Text: ", text, " | Prediction:", pred," | Confidence: ", confidence, "| Ground Truth:", gt)


check_performance(model, tokenizer, cases, fn = predict_em)

Text:  I am not interested in your insurance product  | Prediction: Not Sure  | Confidence:  0 | Ground Truth: Customer
Text:  Can you explain the benefits of FWD insurance?  | Prediction: Not Sure  | Confidence:  0 | Ground Truth: Customer
Text:  FWD seems like a reliable company.  | Prediction: Not Sure  | Confidence:  0 | Ground Truth: Customer
Text:  Tell me more about FWD's customer service.  | Prediction: Not Sure  | Confidence:  0 | Ground Truth: Customer


In [53]:
sample_texts = [
    "Have you heard of FWD before?",
    "I work at FWD insurance",
    "FWD offers a wide range of insurance products to suit your needs.",
    "Our policies are designed to provide comprehensive coverage at competitive rates.",
    "Would you like to know more about our life insurance options?",
    "FWD has been in the insurance industry for over a decade, serving millions of customers.",
    "We pride ourselves on our customer-centric approach and innovative digital solutions.",
]

# Inference and accuracy check
error_count = 0
for sample_text in sample_texts:
    pred = predict_em(sample_text, model, tokenizer)
    gt = "Agent"  # Assuming all samples are "Agent" class
    error = pred != gt 
    if error:
        error_count += 1
        print("Text: ", sample_text, " | Prediction:", pred, "| Ground Truth:", gt)

print(f"Total errors: {error_count}")
error_rate = error_count / len(sample_texts)
print(f"Error Rate: {error_rate:.4f}")

Text:  Have you heard of FWD before?  | Prediction: Customer | Ground Truth: Agent
Text:  I work at FWD insurance  | Prediction: Customer | Ground Truth: Agent
Total errors: 2
Error Rate: 0.2857


The text: 'Have you heard of FWD before?'
Predicted label: Customer
Confidence: 0.76


In [21]:
import torch
from torch.utils.data import Dataset

class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label)
        }

# Create an instance of the dataset
train_dataset = TextClassificationDataset(
    texts=train_dataset["text"],
    labels=train_dataset["label"],
    tokenizer=tokenizer,
    max_length=200
)

# Print some information about the dataset
print(f"Number of samples in the dataset: {len(train_dataset)}")
print(f"Sample from the dataset: {train_dataset[0]}")


TypeError: list indices must be integers or slices, not str

NameError: name 'texts' is not defined